In [122]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import chi2_contingency
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix

In [123]:
credit_data = pd.read_excel ('default of credit card clients.xls') 
credit_data.shape 
#this shows it a large data set. 
# to reduce the calculation time I will randomly select 200 rows instead of 30.000 
# I will do this after the cleaning 

(30001, 25)

In [124]:
credit_data.head(2)

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1


In [125]:
credit_data.columns = list(credit_data.iloc[0])
credit_data_2 = credit_data[1:]
print(credit_data_2.columns)
credit_data_2.head(2) # To make from the first row the column names

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1


In [126]:
# prediction plan: 
# I want to predict on categorical values the bin on limit_bal 
# to do that I have to make bins of: LIMIT_BAL and AGE
# and make categories of SEX, EDUCATION, MARRIAGE and the bins of LIMIT_BAL and AGE

In [127]:
# Create a new data frame for the model 
credit_data_model = credit_data_2[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE']]
credit_data_model.head(2)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE
1,20000,2,2,1,24
2,120000,2,2,2,26


In [128]:
credit_data_model[['LIMIT_BAL','AGE']]= credit_data_model[['LIMIT_BAL','AGE']].apply(pd.to_numeric,errors='coerce')
print(credit_data_model.dtypes)
credit_data_model_1 = credit_data_model

LIMIT_BAL     int64
SEX          object
EDUCATION    object
MARRIAGE     object
AGE           int64
dtype: object


/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [129]:
'''credit_data_3["isnull"] = credit_data_3.isnull().any(axis=1)
missing_values = credit_data_3[credit_data_3['isnull'] == True]
missing_values '''
credit_data_model_1.isna().sum()# there are no missing values 

LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
dtype: int64

In [130]:
credit_data_model_1.groupby(['SEX', 'MARRIAGE','EDUCATION']).count()
# sex = 1,2
# mariage = 0,1,2
# education = 0,1,2,3,4,5,6

LIMIT_BAL   AGE
SEX MARRIAGE EDUCATION                 
1   0        1                  1     1
             2                  1     1
             3                 12    12
    1        0                  2     2
             1               1690  1690
             2               2370  2370
             3               1048  1048
             4                 18    18
             5                 48    48
             6                 14    14
    2        0                  6     6
             1               2633  2633
             2               2940  2940
             3                894   894
             4                 23    23
             5                 46    46
             6                 11    11
    3        1                 30    30
             2                 63    63
             3                 36    36
             4                  1     1
             5                  1     1
2   0        1                  3     3
             2                  5     5
             3                 32    32
    1        0                  2     2
             1               2032  2032
             2               4472  4472
             3               1813  1813
             4                 34    34
             5                102   102
             6                 14    14
    2        0                  4     4
             1               4176  4176
             2               4080  4080
             3               1015  1015
             4                 45    45
             5                 81    81
             6                 10    10
    3        1                 20    20
             2                 99    99
             3                 67    67
             4                  2     2
             5                  2     2
             6                  2     2

In [131]:
credit_data_model_1['AGE_binned'] = pd.cut(credit_data_model_1['AGE'], 3, labels=["20-40", "40-60", "60-80"],)
credit_data_model_1.groupby('AGE_binned').count()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE
AGE_binned,,,,,
20-40,21726,21726,21726,21726,21726
40-60,7935,7935,7935,7935,7935
60-80,339,339,339,339,339


In [132]:
credit_data_model_1['LIMIT_BAL_binned'] = pd.cut(credit_data_model_1['LIMIT_BAL'], 3, labels=["low", "medium", "high"],)
credit_data_model_1.groupby('LIMIT_BAL_binned').mean()


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LIMIT_BAL,AGE
LIMIT_BAL_binned,,
low,132120.236256,35.146335
medium,424740.990991,37.950169
high,730277.777778,41.138889


In [133]:
credit_data_model_2 = credit_data_model_1[['SEX', 'EDUCATION', 'MARRIAGE','AGE_binned', 'LIMIT_BAL_binned']]


In [134]:
credit_data_fixed = pd.get_dummies(credit_data_model_2, drop_first=True)
credit_data_fixed

,SEX_2,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,AGE_binned_40-60,AGE_binned_60-80,LIMIT_BAL_binned_medium,LIMIT_BAL_binned_high
1,1,0,1,0,0,0,0,1,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,1,0,0,0,0,0
3,1,0,1,0,0,0,0,0,1,0,0,0,0,0
4,1,0,1,0,0,0,0,1,0,0,0,0,0,0
5,0,0,1,0,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,0,0,0,1,0,0,0,1,0,0,0,0,0,0
29997,0,0,0,1,0,0,0,0,1,0,1,0,0,0
29998,0,0,1,0,0,0,0,0,1,0,0,0,0,0
29999,0,0,0,1,0,0,0,1,0,0,1,0,0,0


In [135]:
from sklearn.model_selection import train_test_split

# Your code here:
y = credit_data_fixed['LIMIT_BAL_binned_high'] # How high the limit ball will be 
X = credit_data_fixed.drop(["LIMIT_BAL_binned_high"], axis=1) #rest of the data to predict the limit ball 
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, shuffle=True)



In [136]:
# Your code here:
clf = RandomForestClassifier() # because we use the default setting () are empthy 
credit_rf = clf.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [137]:
y_pred_test_rf = credit_rf.predict(X_test)
len(y_pred_test_rf)

6000

In [138]:
X_test.head(2)

,SEX_2,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,AGE_binned_40-60,AGE_binned_60-80,LIMIT_BAL_binned_medium
24643,0,0,1,0,0,0,0,0,1,0,0,0,0
29267,0,0,1,0,0,0,0,0,1,0,0,0,0


In [139]:
y_test.head(2)

24643    0
29267    0
Name: LIMIT_BAL_binned_high, dtype: uint8

In [140]:
# Your code here:
y_pred_test_rf = credit_rf.predict(X_test)
confusion_matrix(y_test, y_pred_test_rf) # compare the axtueal Y_test value with the predicted y values 
# this matrix only works if its categorical 

array([[5992,    0],
       [   8,    0]])